<a href="https://colab.research.google.com/github/alessandrotofani/Tesi_magistrale/blob/master/6_Federated_other_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
!pip install --quiet tensorflow==2.3.0
!pip install --quiet tensorflow_federated==0.17.0
!pip install --quiet --upgrade nest_asyncio

     |████████████████████████████████| 320.4MB 35kB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 460kB 49.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 522kB 7.7MB/s 
     |████████████████████████████████| 112kB 16.1MB/s 
     |████████████████████████████████| 1.1MB 16.7MB/s 
     |████████████████████████████████| 174kB 52.5MB/s 
     |████████████████████████████████| 3.0MB 35.6MB/s 
     |████████████████████████████████| 153kB 42.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [ ]:
import nest_asyncio
nest_asyncio.apply()
%load_ext tensorboard

In [ ]:
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd 
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys 
sys.path.append('/content/drive/MyDrive/Tesi_magistrale/Tesi_magistrale')
import mf

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Import data

I dati vengono importati e poi splittati in train e test. 

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Tesi_magistrale/Dataset/Dalpozz/creditcard.csv')

In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
data = pd.DataFrame(min_max_scaler.fit_transform(data), columns=data.columns)

In [ ]:
col_name = mf.get_col(data)

In [ ]:
print('Rate safe/fraud:', (mf.ratio(data)).round(3))

Rate fraud/safe: 0.036


In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.1)

Smote: https://imbalanced-learn.org/stable/generated/imblearn.over_sampling.SMOTE.html

RandomUnderSampler: https://imbalanced-learn.org/stable/generated/imblearn.under_sampling.RandomUnderSampler.html

In [ ]:
def underSampling(data, frac_under=0.1):
  from imblearn.under_sampling import RandomUnderSampler 
  us = RandomUnderSampler(sampling_strategy=frac_under, random_state=42)
  y = data['Class']
  X = data.drop(columns = ['Class'])
  X_us, y_us = us.fit_resample(X, y)
  return X_us, y_us

def overSampling(X, y, frac_over=0.01):
  from imblearn.over_sampling import SMOTE
  sm = SMOTE(sampling_strategy=frac_over, random_state=42)
  X_sm, y_sm = sm.fit_resample(X, y)  
  return X_sm, y_sm

def mergeResult(X, y, col_name):
  y_res = np.ndarray(shape=(np.shape(y)[0],1), buffer = y)
  data = np.concatenate((X,y_res), axis = 1)
  col_name.append('Class')
  dataset = pd.DataFrame(data=data, columns=col_name)  
  return dataset

In [ ]:
# X, y = underSampling(train_data)

In [ ]:
X, y = overSampling(train_data)

In [ ]:
train_data = mergeResult(X, y, col_name)

In [ ]:
print('Rate safe/fraud:', (mf.ratio(train_data)).round(3))
print('Dataset size:', train_data.shape[0])

Rate fraud/safe: 0.25
Dataset size: 241644


In [ ]:
del data, X, y

# Convert data

Il dataset deve essere convertito in un tensore, con componenti (feature_vector, label). 

In [ ]:
def to_tensor(data, n_clients = 5):
  shuffled = data.sample(frac=1)
  result = np.array_split(shuffled, n_clients)  

  res = []

  new_res = []
  label = []

  for dataset in result:
    res.append(mf.feature_scaling(dataset))
  
  for subset in res:
    label.append(subset['isFraud'])
    new_res.append(subset.drop(columns = ['isFraud']).to_numpy())

  dataset = {}
  for i in range(n_clients):
    dataset[i] = tf.data.Dataset.from_tensor_slices((new_res[i], label[i]))
  return dataset

In [ ]:
dataset = to_tensor(train_data)
test_set = to_tensor(test_data)

In [ ]:
del train_data, test_data

# Federated data

Si definisce la funzione di preprocessing del dataset, che serve a creare l'OrderedDict, su cui si andranno a creare le batch necessarie per il training del modello. 

In [ ]:
NUM_CLIENTS = 5
NUM_EPOCHS = 10
BATCH_SIZE = 500
SHUFFLE_BUFFER = 10
PREFETCH_BUFFER = 10

def preprocess(dataset):
  def batch_format_fn(e1, e2):
    return collections.OrderedDict(
        x = tf.cast(e1, tf.float32),
        y = tf.cast(e2, tf.int32))
  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(
      BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)

preprocessed_example_dataset = preprocess(dataset[0])
# sample_batch = tf.nest.map_structure(lambda x: x.numpy(), next(iter(preprocessed_example_dataset)))
# sample_batch

I dati federati sono una lista di dataset divisi per cliente. 

In [ ]:
def make_federated_data(dataset):
  federated = []
  for i in dataset:
    federated.append(preprocess(dataset[i]))
  return federated

federated_train_data = make_federated_data(dataset)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))

Number of client datasets: 5
First dataset: <PrefetchDataset shapes: OrderedDict([(x, (None, 1103)), (y, (None,))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>


In [ ]:
del dataset

# Model creation and training

Creazione della rete neurale che sarà trainata. 

Viene anche definita la model function, in cui si specifica il modello, il tipo di input, la loss e le metriche da utilizzare. 

Infine si costruisce il processo di averaging, specificando l'optimizer da usare, cioè SGD, e il learning rate del server e del client. 

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input

def create_keras_model():
  model = Sequential()
  model.add(Input(shape=(1103,)))  
  model.add(Dense(50, activation='relu')) 
  model.add(Dropout(0.75))
  return model

def model_fn():
  soglia = 0.5
  keras_model = create_keras_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=preprocessed_example_dataset.element_spec,
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=[tf.keras.metrics.BinaryAccuracy(), 
               tf.keras.metrics.Recall(thresholds=soglia),
               tf.keras.metrics.Precision(thresholds=soglia)])
  
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,  
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02), #0.4
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.4)) #0.8

Training del modello. 

Gpu usage: https://colab.research.google.com/notebooks/gpu.ipynb#scrollTo=Y04m-jvKRDsJ

In [ ]:
NUM_ROUNDS = 21
# with tf.device('/device:GPU:0'):
state = iterative_process.initialize()
state, metrics = iterative_process.next(state, federated_train_data)
print('round  1, metrics={}'.format(metrics))
for round_num in range(2, NUM_ROUNDS):
  state, metrics = iterative_process.next(state, federated_train_data)
  print('round {:2d}, metrics={}'.format(round_num, metrics))

# Model evaluation

Evaluation del modello sui test data. 

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_fn)
train_metrics = evaluation(state.model, federated_train_data)

In [ ]:
federated_test_data = make_federated_data(test_set)

In [ ]:
test_metrics = evaluation(state.model, federated_test_data)
str(test_metrics)

Board di tensorboad, per visualizzare la loss e le metriche in modo interattivo. 

In [ ]:
# logdir = "/tmp/logs/scalars/training/"
# summary_writer = tf.summary.create_file_writer(logdir)
# state = iterative_process.initialize()
# with summary_writer.as_default():
#   for round_num in range(1, NUM_ROUNDS):
#     state, metrics = iterative_process.next(state, federated_train_data)
#     for name, value in metrics['train'].items():
#       tf.summary.scalar(name, value, step=round_num)

In [ ]:
# !ls {logdir}
# %tensorboard --logdir {logdir} --port=0